In [57]:
import pandas as pd
import chardet
import os
import re
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pandas import Period
import numpy as np

pd.set_option("display.max_columns", None)

### Load data

In [58]:
df = pd.read_csv("../data/all_data.csv", dtype={"AI summary": str, "EAN": str}, low_memory=False)
df = df.sort_values(["extract_date", "COGS"], ascending=False)
print(df.shape)
df.head(3)

(34147, 77)


,ASIN,Nom du produit,Marque,COGS,COGS LM (%),COGS SPLY (%),Unités,Unités LM (%),Unités SPLY (%),Retours,Retours LM (%),Retours SPLY (%),Source (Fabrication ou Approvisionnement),is_in_vendor_central,Item no.,EAN,Item,Extras,Size,Description Item,Cat. 1,Cat. 2,Cat. 3,Cat. 4,Visible,Title Content,Variations,Brand store URL,Images,Videos,AI summary,Code,Catalogue,Ads Impressions CM,Ads Clicks CM,Ads CTR CM,Ads Units 14d CM,Ads Costs CM,Ads RoAS CM,Ads CVR CM,Sell-out CM,Sell-out PM,Total sell-out CM,Total sell-out PM,Units CM,Units PM,SRP,Margin,Replacements CM,Replacements PM,Stock,Stock value,Total stock,Total stock value,Reviews,Avg. stars,Stars,Coverage,Rank 1,Rank 2,Buy Box,Buy Box PM,Days not Buy Box,Views CM,Views PM,CVR CM,CVR PM,is_in_amvisor,Net PPM,Net PPM SPLY,extract_date,annee,mois,COGS SPLY (€),COGS diff SPLY (€),Category 1,Category 2
23,B00005QFCJ,Hama Adaptateur Cassette VHS-C/VHS (Convertiss...,Hama,10142.55,-7.30,63.37,495.0,-7.30,63.37,64.0,-31.91,45.45,1_fab,True,00044704,4007249447043,Hama Cassette adaptatrice VHS-C/VHS motorisée,NaN,NaN,REGULAR,Photo & Accessories,Studio & Diverse,Camera,Adapters,Yes,Hama Adaptateur Cassette VHS-C/VHS (Convertiss...,B00005QFCJ,NaN,13.0,2.0,Les clients apprécient le bon rapport qualité-...,PR,STO,66234.0,1550.0,2.3402,225.0,200.90,27.36,14.52,0.00,0.00,9302.46,10941.66,0.0,0.0,41.99,19.8,0.0,0.0,0.0,0.0,293.0,6003.57,901.0,4.0,3926.0,NaN,1304.0,221.0,Amazon,100.0,NaN,8037.0,8960.0,5.65,5.96,True,39.35,42.289092,2025-03,2025,3,6208.47,3934.08,Photo & Accessories,Studio & Diverse
82,B00006JAQ7,"Hama 007103 - Ruban photo transparent, Blanc, ...",Hama,3787.44,105.46,851.83,1313.0,102.94,1041.74,11.0,1000.00,NaN,1_fab,True,00007103,4007249071033,Hama Distributeur de bandes adhésives,NaN,2 x 500 bandes,REGULAR,Picture World,Storage & Design,Photo Storage & Presentation,Storage & Archiving,Yes,"Hama 007103 - Ruban photo transparent, Blanc, ...",B00005YVIU B00006JAQ7 B0CPF2BMJL,NaN,5.0,2.0,Les clients apprécient la super adhérence des ...,PR,STO,90395.0,1144.0,1.2656,552.0,526.34,4.13,48.25,884.83,394.44,3509.31,1843.40,305.0,136.0,3.99,68.1,0.0,0.0,356.0,1032.4,786.0,2278.34,2000.0,4.6,12826.0,1.1,398.0,1.0,Amazon,58.8,NaN,2642.0,1730.0,46.06,37.40,True,35.85,52.236631,2025-03,2025,3,397.91,3389.53,Picture World,Storage & Design
32,B00005YVIU,"Hama Distributeurs Coins photo (1000 coins, au...",Hama,3035.01,114.29,7188.69,1044.0,113.93,8600.00,9.0,NaN,NaN,1_fab,True,00007108,4007249071088,Hama Boîte distributrice coins adhésifs pr photos,Transparent,2x500 soins,REGULAR,Picture World,Storage & Design,Photo Storage & Presentation,Storage & Archiving,Yes,"Hama Distributeurs Coins photo (1000 coins, au...",B00005YVIU B00006JAQ7 B0CPF2BMJL,NaN,7.0,1.0,Les clients apprécient la super adhérence des ...,PR,PUA,184208.0,1878.0,1.0195,738.0,1144.90,2.76,39.30,1058.50,124.70,2834.91,1416.30,365.0,43.0,3.99,70.2,0.0,0.0,167.0,484.3,552.0,1600.80,2002.0,4.6,12827.0,0.4,273.0,1.0,Amazon,58.8,NaN,2494.0,1249.0,39.09,39.07,True,38.82,54.560787,2025-03,2025,3,41.64,2993.37,Picture World,Storage & Design


### Cheatsheet

Top products list

In [59]:
dt = df

dt['extract_date'] = pd.to_datetime(dt['extract_date'], format='%Y-%m')
dt['year'] = dt['extract_date'].dt.year
dt['quarter'] = dt['extract_date'].dt.quarter
dt['year_quarter'] = df['year'].astype(str) + '-Q' + dt['quarter'].astype(str)

dt = dt.groupby(["year_quarter", "ASIN"], dropna=False).agg({"COGS": "sum", "Unités": "sum"}).reset_index().sort_values(["year_quarter", "COGS"], ascending=[False, False]).reset_index()

dt_Q12025 = dt[dt["year_quarter"] == "2025-Q1"]
dt_Q12024 = dt[dt["year_quarter"] == "2024-Q1"]

# On renomme d'abord les colonnes pour chaque dataframe
dt_Q12024 = dt_Q12024.rename(columns={
    "Product": "ASIN",
    "Unités": "QTY YTQ1 '24",
    "COGS": "COGS YTQ1 '24"
})

dt_Q12025 = dt_Q12025.rename(columns={
    "Product": "ASIN",
    "Unités": "QTY YTQ1 '25",
    "COGS": "COGS YTQ1 '25"
})

dt_Q12025.drop(["index"], inplace=True, axis=1)
dt_Q12024.drop(["index"], inplace=True, axis=1)

merged = pd.merge(dt_Q12024[['ASIN', "QTY YTQ1 '24", "COGS YTQ1 '24"]],
                  dt_Q12025[['ASIN', "QTY YTQ1 '25", "COGS YTQ1 '25"]],
                  on='ASIN',
                  how='outer')

# Calcul du % de changement sur les quantités
merged["% Change QTY"] = ((merged["QTY YTQ1 '25"] - merged["QTY YTQ1 '24"]) / merged["QTY YTQ1 '24"].replace(0, np.nan)) * 100
merged["% Change COGS"] = ((merged["COGS YTQ1 '25"] - merged["COGS YTQ1 '24"]) / merged["COGS YTQ1 '24"].replace(0, np.nan)) * 100
merged["€ Change QTY"] = ((merged["QTY YTQ1 '25"] - merged["QTY YTQ1 '24"])).replace(0, np.nan)
merged["€ Change COGS"] = ((merged["COGS YTQ1 '25"] - merged["COGS YTQ1 '24"])).replace(0, np.nan)
merged = merged.round(0)

print(f"Sort by Q1 2025 performance (COGS YTQ1 '25)")
merged = merged.sort_values("COGS YTQ1 '25", ascending=False)
top_products = merged.ASIN.unique()
top_products

Sort by Q1 2025 performance (COGS YTQ1 '25)


array(['B00005QFCJ', 'B00D2S92L0', 'B00006JAQ7', ..., 'B09GLG4LW1',
       'B0BFR5WYF6', 'B0BL1S7WB7'], dtype=object)

Item

In [60]:
dt = df[df["ASIN"].isin(top_products)][["ASIN", "Nom du produit", "Cat. 1", "Cat. 2", "Cat. 3", "Cat. 4", "Category 1", "Category 2"]].drop_duplicates()
dt

,ASIN,Nom du produit,Cat. 1,Cat. 2,Cat. 3,Cat. 4,Category 1,Category 2
23,B00005QFCJ,Hama Adaptateur Cassette VHS-C/VHS (Convertiss...,Photo & Accessories,Studio & Diverse,Camera,Adapters,Photo & Accessories,Studio & Diverse
82,B00006JAQ7,"Hama 007103 - Ruban photo transparent, Blanc, ...",Picture World,Storage & Design,Photo Storage & Presentation,Storage & Archiving,Picture World,Storage & Design
32,B00005YVIU,"Hama Distributeurs Coins photo (1000 coins, au...",Picture World,Storage & Design,Photo Storage & Presentation,Storage & Archiving,Picture World,Storage & Design
695,B01N2RIZMU,"Xavax Socle universel ""Gigant"" (60 x 60 cm, au...",Home & Office,Domestic Appliance,Home,Cleaning,Home & Office,Domestic Appliance
540,B00D2S92L0,Xavax Hama Boîte de transport pour un gâteau (...,Home & Office,Housewares,Cooking & Dining,Tableware,Home & Office,Housewares
...,...,...,...,...,...,...,...,...
34059,B0B636GVMW,"Hama Radio numérique DIR3300SBT (Spotify, WLAN...",NaN,NaN,NaN,NaN,Audio,Digital Radio
34076,B0BL1C6KGC,Hama Support Téléphone Voiture Magnet (Support...,NaN,NaN,NaN,NaN,Mobilcom,Brackets
34092,B0BL1SYKTW,Hama Câble USB-C Extreme (Câble de charge USB-...,NaN,NaN,NaN,NaN,Mobilcom,Cables
34103,B0BL1VX724,Hama Chargeur Secteur USB-C (Bloc Secteur USB-...,NaN,NaN,NaN,NaN,Mobilcom,Power Chargers


In [61]:
dt[(dt['Cat. 1']=="Distribution Brand") & (dt['Cat. 2']=="Koss")]

,ASIN,Nom du produit,Cat. 1,Cat. 2,Cat. 3,Cat. 4,Category 1,Category 2
1288,B00001P4ZH,NaN,Distribution Brand,Koss,Audio,Headphones/Headsets,-,-
1303,B000O2KIMO,NaN,Distribution Brand,Koss,Audio,Headphones/Headsets,-,-
1421,B004XUMEMU,NaN,Distribution Brand,Koss,Audio,Headphones/Headsets,-,-


In [62]:
grouped = (
    dt.groupby(['Category 1', 'Category 2', 'Cat. 1', 'Cat. 2'])
    .agg({'Nom du produit': lambda x: list(x)[:3]})  # Prendre les 3 premiers ASIN
    .reset_index()
)

# Renommer la colonne pour plus de clarté
grouped.rename(columns={"Nom du produit": "top_3_asin"}, inplace=True)
grouped.to_csv("../data/cheatsheet.csv", index=False)

In [65]:
df[df["Nom du produit"] == """Hama "Freedom Light Ecouteurs Bluetooth® sans Fil USB C - 7H d'Autonomie, Assistant Vocaux Siri & Google, Microphone Intégré, Commande Tactile - Rouge"""]

,ASIN,Nom du produit,Marque,COGS,COGS LM (%),COGS SPLY (%),Unités,Unités LM (%),Unités SPLY (%),Retours,Retours LM (%),Retours SPLY (%),Source (Fabrication ou Approvisionnement),is_in_vendor_central,Item no.,EAN,Item,Extras,Size,Description Item,Cat. 1,Cat. 2,Cat. 3,Cat. 4,Visible,Title Content,Variations,Brand store URL,Images,Videos,AI summary,Code,Catalogue,Ads Impressions CM,Ads Clicks CM,Ads CTR CM,Ads Units 14d CM,Ads Costs CM,Ads RoAS CM,Ads CVR CM,Sell-out CM,Sell-out PM,Total sell-out CM,Total sell-out PM,Units CM,Units PM,SRP,Margin,Replacements CM,Replacements PM,Stock,Stock value,Total stock,Total stock value,Reviews,Avg. stars,Stars,Coverage,Rank 1,Rank 2,Buy Box,Buy Box PM,Days not Buy Box,Views CM,Views PM,CVR CM,CVR PM,is_in_amvisor,Net PPM,Net PPM SPLY,extract_date,annee,mois,COGS SPLY (€),COGS diff SPLY (€),Category 1,Category 2,year,quarter,year_quarter
1025,B096XM5NMX,"Hama ""Freedom Light Ecouteurs Bluetooth® sans ...",Hama,NaN,NaN,-100.00,NaN,NaN,-100.0,NaN,NaN,NaN,1_fab,True,00184075,4047443458957,"Hama Écout. Bluetooth ""Freedom Light"", Tr. Wir...",Rouge,NaN,REGULAR,Picture World,Album,Audio,Headphones/Headsets,Yes,"Hama ""Freedom Light Ecouteurs Bluetooth® sans ...",B096XH84CL B096XK8FJ3 B096XM2WGL B096XM5NMX B0...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,19.99,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.4,25.0,NaN,797738.0,913.0,Amazon,0.0,NaN,NaN,NaN,NaN,NaN,True,0.00,0.000000,2025-03-01,2025,3,17.88,NaN,Audio,Headphone/Headsets,2025,1,2025-Q1
6871,B096XM5NMX,"Hama ""Freedom Light Ecouteurs Bluetooth® sans ...",Hama,NaN,-100.0,NaN,NaN,-100.0,NaN,NaN,NaN,NaN,1_fab,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.00,0.000000,2025-01-01,2025,1,NaN,NaN,Audio,Headphone/Headsets,2025,1,2025-Q1
9401,B096XM5NMX,"Hama ""Freedom Light Ecouteurs Bluetooth® sans ...",Hama,17.82,NaN,-0.34,1.0,NaN,0.0,NaN,NaN,NaN,1_fab,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,27.09,27.907695,2024-12-01,2024,12,NaN,NaN,Audio,Headphone/Headsets,2024,4,2024-Q4
14486,B096XM5NMX,"Hama ""Freedom Light Ecouteurs Bluetooth® sans ...",Hama,NaN,-100.0,NaN,NaN,-100.0,NaN,NaN,NaN,NaN,1_fab,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.00,0.000000,2024-10-01,2024,10,NaN,NaN,Audio,Headphone/Headsets,2024,4,2024-Q4
16975,B096XM5NMX,"Hama ""Freedom Light Ecouteurs Bluetooth® sans ...",Hama,17.88,NaN,0.00,1.0,NaN,0.0,NaN,NaN,NaN,1_fab,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,11.40,13.410187,2024-09-01,2024,9,NaN,NaN,Audio,Headphone/Headsets,2024,3,2024-Q3
18292,B096XM5NMX,"Hama ""Freedom Light Ecouteurs Bluetooth® sans ...",Hama,NaN,-100.0,-100.00,NaN,-100.0,-100.0,NaN,NaN,NaN,1_fab,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.00,0.000000,2024-08-01,2024,8,NaN,NaN,Audio,Headphone/Headsets,2024,3,2024-Q3
20838,B096XM5NMX,"Hama ""Freedom Light Ecouteurs Bluetooth® sans ...",Hama,17.88,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1_fab,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1